In [8]:
import sys
sys.path.append("./Incontext-learning") # this part works for goole colab

In [1]:
import transformers

In [9]:
from tinytransformer import LinearNanoGPT

In [11]:
! nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


tinytransformer.LinearNanoGPT